In [38]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib as plt
import tensorflow as tf
import pickle

# Downloading MNIST dataset

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

# Split Data

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(48000, 28, 28) (48000,)
(12000, 28, 28) (12000,)
(10000, 28, 28) (10000,)


In [6]:
for i in range(0, 10):
    print("digit " + str(i) + " - " + str(len(np.where(y_train == i)[0])))

digit 0 - 4733
digit 1 - 5415
digit 2 - 4801
digit 3 - 4893
digit 4 - 4656
digit 5 - 4322
digit 6 - 4769
digit 7 - 5047
digit 8 - 4665
digit 9 - 4699


# Preprocessing

In [9]:
def preprocessing(img):
    img = cv2.equalizeHist(img)
    img = img/225
    return img

In [10]:
X_train_preprocessed = np.array(list(map(preprocessing, X_train)))
X_val_preprocessed = np.array(list(map(preprocessing, X_val)))
X_test_preprocessed = np.array(list(map(preprocessing, X_test)))

In [11]:
X_train_reshaped = X_train_preprocessed.reshape(X_train_preprocessed.shape[0], X_train_preprocessed.shape[1], X_train_preprocessed.shape[2], 1)
X_val_reshaped = X_val_preprocessed.reshape(X_val_preprocessed.shape[0], X_val_preprocessed.shape[1], X_val_preprocessed.shape[2], 1)
X_test_reshaped = X_test_preprocessed.reshape(X_test_preprocessed.shape[0], X_test_preprocessed.shape[1], X_test_preprocessed.shape[2], 1)

In [16]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range =0.1,
                                                          height_shift_range = 0.1,
                                                          zoom_range = 0.2,
                                                          shear_range = 0.1,
                                                          rotation_range = 10)

data_gen.fit(X_train_reshaped)

In [35]:
y_train_encoded = tf.keras.utils.to_categorical(y_train, 10)
y_val_encoded = tf.keras.utils.to_categorical(y_val, 10)
y_test_encoded = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [32]:
model = tf.keras.models.Sequential()

# Convolutional layers to extract spatial features
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten the output from the convolutional layers
model.add(tf.keras.layers.Flatten())

# Fully connected (dense) layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))

# Output layer with 10 classes for digit classification
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the model with a specific learning rate
learning_rate = 0.001  # You can tune this value
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 3, 3, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,506 (541.04 KB)

 Trainable params: 138,506 (541.04 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
history = model.fit(
    data_gen.flow(X_train_reshaped, y_train_encoded, batch_size=50),
    steps_per_epoch=2000,
    epochs=10,
    validation_data=(X_val_reshaped, y_val_encoded),
    shuffle=True
)

Epoch 1/10
  10/2000 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.1258 - loss: 2.2806 

/Users/hasangaranasinghe/Documents/Projects/Text-Detection/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 960/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.7680 - loss: 0.6942

2024-08-16 08:21:43.902196: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8274 - loss: 0.5225 - val_accuracy: 0.9764 - val_loss: 0.0716
Epoch 2/10
 956/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9561 - loss: 0.1403

...

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9584 - loss: 0.1334 - val_accuracy: 0.9764 - val_loss: 0.0766
Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9686 - loss: 0.0995 - val_accuracy: 0.9851 - val_loss: 0.0476
Epoch 4/10
 960/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9746 - loss: 0.0819

2024-08-16 08:22:18.899626: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9751 - loss: 0.0803 - val_accuracy: 0.9852 - val_loss: 0.0489
Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9785 - loss: 0.0688 - val_accuracy: 0.9860 - val_loss: 0.0448
Epoch 6/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9813 - loss: 0.0600 - val_accuracy: 0.9878 - val_loss: 0.0361
Epoch 7/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9820 - loss: 0.0573 - val_accuracy: 0.9894 - val_loss: 0.0329
Epoch 8/10
 956/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9839 - loss: 0.0527

2024-08-16 08:23:03.864226: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9840 - loss: 0.0521 - val_accuracy: 0.9886 - val_loss: 0.0437
Epoch 9/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9851 - loss: 0.0495 - val_accuracy: 0.9903 - val_loss: 0.0331
Epoch 10/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9859 - loss: 0.0470 - val_accuracy: 0.9890 - val_loss: 0.0345


In [36]:
score = model.evaluate(X_test_reshaped, y_test_encoded, verbose=0)
score

[0.03134825453162193, 0.9904000163078308]

In [39]:
pickle_out = open("../model_trained.p", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()